# gene-counts-rsem-expected_count-collapsed-deseq 
parquet file = 12G

In [2]:
import numpy as np
import polars as pl
pl.Config.set_fmt_str_lengths(100)
import pandas as pd
pd.set_option('display.max_columns', None)
import pyarrow 

import sys
sys.path.insert(0, '/home/stearb/U24/code/utils/')
from utils import drop_single_unique_value_cols, fill_missing_cols

# main path to data
path='/mnt/isilon/dbhi_bfx/MTP/OpenPedCan_Files/v12/tabular/'
file='gene-counts-rsem-expected_count-collapsed-deseq.tsv.gz'
pfile='/parquet/gene-counts-rsem-expected_count-collapsed-deseq.parquet'


!ls -lah /mnt/isilon/dbhi_bfx/MTP/OpenPedCan_Files/v12/tabular/parquet/gene-counts-rsem-expected_count-collapsed-deseq.parquet

flatpak: symbol lookup error: /lib64/librpmio.so.9: undefined symbol: lzma_stream_encoder_mt, version XZ_5.2
-rw-r--r-- 1 stearb reslnusers 12G Dec 22 09:11 /mnt/isilon/dbhi_bfx/MTP/OpenPedCan_Files/v12/tabular/parquet/gene-counts-rsem-expected_count-collapsed-deseq.parquet


In [1]:
mkdir /mnt/isilon/dbhi_bfx/MTP/OpenPedCan_Files/v12/tabular/notebooks/

In [3]:
cp /home/stearb/U24/data/helper_files/hgnc_master.txt /mnt/isilon/dbhi_bfx/MTP/OpenPedCan_Files/v12/tabular/

# Function/Code for Binning expression values
### pl.series.cut()
https://stackoverflow.com/questions/75229981/how-to-use-polars-cut-method-returning-result-to-original-df
    

In [2]:


def bin_column(df, col2bin='Disease_MeanTpm'):
    '''works with polars df'''
    
    tpm_bins = list([0.0000000,7e-4,8e-4,9e-4]) + list(np.linspace(1e-3,9e-3,9)) + \
               list(np.round(np.linspace(1e-2,9e-2,9),2)) + list(np.round(np.linspace(.1,1,10),2)) + \
               list(np.linspace(2,100,99)) + list(np.arange(100,1100,100)[1:]) +  \
                list(np.arange(2000,11000,1000)) + list(np.arange(20_000,110_000,10_000)) + [300_000] 

    # check that none of the disease expression values are out of range, ie larger than the max bin value
    # use for pandas
    #if np.sum(df['Disease_MeanTpm'] > np.max(tpm_bins)) == 0:
        
    # use for polars
    if len(df.filter(df[col2bin] > np.max(tpm_bins))) == 0:
        df_binned = df.with_columns(  df[col2bin].cut(breaks=tpm_bins).alias('bins')  )

        # check that order has been maintained
        #assert pl.all(df[col2bin] == df_binned[col2bin])

    else:
        print('OUT OF RANGE ERROR!')
        assert False
        
    return df_binned

In [3]:
hgnc_master = pl.read_csv('/home/stearb/U24/data/helper_files/hgnc_master.txt',
                          infer_schema_length=10000,separator='\t')
hgnc_master.head()

hgnc_select_sym = hgnc_master[['hgnc_id','symbol']].rename({'symbol':'Gene_symbol'})
hgnc_select_sym

hgnc_select_ens = hgnc_master[['hgnc_id','ensembl_gene_id']].rename({'ensembl_gene_id':'targetFromSourceId'})
hgnc_select_ens.head(3)

hgnc_id,targetFromSourceId
str,str
"""HGNC:5""","""ENSG00000121410"""
"""HGNC:37133""","""ENSG00000268895"""
"""HGNC:24086""","""ENSG00000148584"""


In [ ]:
# DONT LOAD ENTIRE FILE, KERNEL WILL CRASH

#genecounts = pl.read_csv(path+file,separator='\t')
#genecounts = pl.read_parquet(path+pfile) #n_rows=10) #, low_memory=True) # use_pyarrow=True, 
#genecounts

In [ ]:
# Look at all unique diseases in df

unique_diseases = pl.read_parquet(path+pfile, columns=['Disease']).unique().to_series().sort().to_numpy()
unique_diseases

In [ ]:
len(uniq)

In [ ]:
# NBL1 TARGET
# NBL2 CBTN
# NBL3 KF
# NBL4 all_cohorts
# make Cohort_ID with these

# ALL might have all_cohorts as well

# bin diseaseMeanTPM values

# for each gene,  the expression for a disease (aka cancer type), in the `Disease_MeanTpm` column 
# is the same for all tissues bc they only measured the gene expression in the tumour. 

# Log2FC is different for each gene-tissue bc the Disease_MeanTpm is compared to the GTEX_normal tissue exp
# levels, which are different for every tissue.

# connect disease 

# every cancer type has an 'all_cohorts', if there is only a single cohort, 
# the all_cohorts tpm and mean values will be identical to the single cohort values

In [ ]:
unique_sources = pl.read_parquet(path+pfile, columns=['datasourceId','Dataset','comparisonId']).unique().to_series().sort().to_numpy()
unique_sources

In [ ]:
q = pl.scan_parquet(path+pfile).select(['datasourceId','Dataset','comparisonId']).unique()
unique_datasource = q.collect()
unique_datasource

In [4]:
%%capture
'''
# Look at just 'Neuroblastoma' Disease data from the 'all_cohorts_vs_GTEx' analysis
# 'Neuroblastoma' # 'Ganglioneuroblastoma' # 'CNS neuroblastoma'

#DISEASE='Neuroblastoma'
DISEASE='Acute Lymphoblastic Leukemia'

# Use scan for lazy_loading
# 
q = pl.scan_parquet(path+pfile).filter((pl.col("Disease") == DISEASE)  & \
                                        (pl.col('datasourceId') == 'all_cohorts_vs_GTEx'))
nbl_gc_all = q.collect()
nbl_gc_all.head() '''

# Start of workflow

In [4]:
%%time

# the expression values were collected in the tumors, not in each tissue
# which means for each disease, all the gene expression values are the same for every tissue. So we
# can just use any tissue and take the Disease_MeanTpm value of that row bc they will be the same for every
# tissue for a disease anyway
# Example:

# DISEASE       GENE    TISSUE              EXP_VALUE   Log2FC
# Neuroblastoma Gene1   Adrenal Gland       0.5           0.1
# Neuroblastoma Gene2   Aorta               0.5            1
# Neuroblastoma Gene3   Thalamus            0.5            10

# THe log2FC will be unique for every gene-tissue row bc its compared to the GTEx tissue exp value for
# each (healthy) tissue. If we are just interested in gene expression in the tumor we can exclude this for now.
# We can still filter by it though...
# So just choose a tissue to filter by, and this will capture the values we need. The schema for this 
# dataset does not include a tissue node. It includes gene - disease - expression nodes all connected to 
# a center MTP metanode.

# Include these filters for a stricter query ---

# 
# (pl.col('log2FoldChange') >= 0.5)         
#             (pl.col('padj') <= 0.05)



cols = ['datasourceId','targetFromSourceId','Gene_symbol','GTEx_subgroup','Disease','MONDO',
        'comparisonId','Disease_MeanTpm', 'log2FoldChange', 'padj']

q = pl.scan_parquet(path+pfile)\
        .filter(
            (pl.col('datasourceId') == 'all_cohorts_vs_GTEx') & \
            (pl.col('GTEx_subgroup') == 'Adipose - Subcutaneous') & \
            (pl.col('Disease_MeanTpm') > 0.0)
        ) \
         .select(cols)

df  = q.collect()

print(df.shape)
#df.head(5)

(1641207, 10)
CPU times: user 13.1 s, sys: 9.36 s, total: 22.5 s
Wall time: 13.9 s


In [9]:
df.select('Disease').unique()#.shape

Disease
str
"""Diffuse intrinsic pontine glioma"""
"""Choroid plexus papilloma"""
"""Meningioma"""
"""Chordoma"""
"""High-grade glioma"""
…
"""Germinoma"""
"""CNS neuroblastoma"""
"""Pilocytic astrocytoma"""


# Bin exp values

In [27]:
df = bin_column(df=df,col2bin='Disease_MeanTpm')

df = df.to_pandas()

assert not df.bins.isna().sum() # check for NaNs in bins col
assert not len([i for i in df['targetFromSourceId'] if '.' in i]) # check for periods in the ENSEMBL CODES


df['MTPEXP_CODEID'] = 'MTPEXP:' + df['Disease'] + '_' + \
                                df['targetFromSourceId']       #+ '_' + df['GTEx_subgroup']  # dont need tissue

df['MTPEXP_CODEID'] = [i.replace(' ','_').replace('MTPEXP_','MTPEXP ') for i in df['MTPEXP_CODEID']]

df = df.drop(['datasourceId','Gene_symbol','GTEx_subgroup', 'comparisonId','log2FoldChange',
                        'padj','Disease','Disease_MeanTpm'],axis=1)

df['targetFromSourceId'] = ['ENSEMBL:'+i for i in df['targetFromSourceId']]
df['MONDO'] = [i.replace('_',':') for i in df['MONDO']]

df['EXPBINS'] = [ 'EXPBINS '+    i.replace('(','')\
                                  .replace(')','')\
                                  .replace('[','')\
                                  .replace(']','')\
                                  .replace('-inf','0.0')\
                                   .replace(',','')\
                                  .replace(' ','.') for i in df['bins']]

df['EXPBINS'] = [i.replace(' ',':') for i in df['EXPBINS']]

df = df.drop(['bins'],axis=1)
df.head()

,targetFromSourceId,MONDO,MTPEXP_CODEID,EXPBINS
0,ENSEMBL:ENSG00000121410,MONDO:0004967,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...,EXPBINS:21.22
1,ENSEMBL:ENSG00000268895,MONDO:0004967,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000002...,EXPBINS:6.7
2,ENSEMBL:ENSG00000175899,MONDO:0004967,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...,EXPBINS:0.7.0.8
3,ENSEMBL:ENSG00000245105,MONDO:0004967,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000002...,EXPBINS:0.6.0.7
4,ENSEMBL:ENSG00000166535,MONDO:0004967,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...,EXPBINS:0.03.0.04


In [28]:

edges_mtp2disease = df[['MTPEXP_CODEID','MONDO']].rename({'MTPEXP_CODEID':'subject',
                                                          'MONDO':'object'},axis=1)
edges_mtp2disease['predicate'] = 'disease'
edges_mtp2disease = edges_mtp2disease[['subject','predicate','object']]


edges_mtp2gene = df[['MTPEXP_CODEID','targetFromSourceId']].rename({'MTPEXP_CODEID':'subject',
                                                                    'targetFromSourceId':'object'},axis=1)
edges_mtp2gene['predicate'] = 'gene'
edges_mtp2gene = edges_mtp2gene[['subject','predicate','object']]


edges_mtp2exp = df[['MTPEXP_CODEID','EXPBINS']].rename({'MTPEXP_CODEID':'subject',
                                                                    'EXPBINS':'object'},axis=1)
edges_mtp2exp['predicate'] = 'tumor_expression' 
edges_mtp2exp = edges_mtp2exp[['subject','predicate','object']]

In [29]:
edges_mtp2disease

,subject,predicate,object
0,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...,disease,MONDO:0004967
1,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000002...,disease,MONDO:0004967
2,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...,disease,MONDO:0004967
3,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000002...,disease,MONDO:0004967
4,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...,disease,MONDO:0004967
...,...,...,...
1641202,MTPEXP:Wilms_tumor_ENSG00000203995,disease,MONDO:0006058
1641203,MTPEXP:Wilms_tumor_ENSG00000162378,disease,MONDO:0006058
1641204,MTPEXP:Wilms_tumor_ENSG00000159840,disease,MONDO:0006058
1641205,MTPEXP:Wilms_tumor_ENSG00000074755,disease,MONDO:0006058


In [46]:
edges = pd.concat([edges_mtp2disease,edges_mtp2gene,edges_mtp2exp]).reset_index(drop=True)

# /home/stearb/U24/data/MTP/
edges.to_csv('/mnt/isilon/opentargets/U24KG/data/owlnets_nodes_and_edges/MTPEXP/OWLNETS_edgelist.txt',
             sep= "\t",index=False)


,node_id
0,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...
1,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000002...
2,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...
3,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000002...
4,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...
...,...
1641202,MTPEXP:Wilms_tumor_ENSG00000203995
1641203,MTPEXP:Wilms_tumor_ENSG00000162378
1641204,MTPEXP:Wilms_tumor_ENSG00000159840
1641205,MTPEXP:Wilms_tumor_ENSG00000074755


In [41]:
nodes = df[['MTPEXP_CODEID']].rename({'MTPEXP_CODEID':'node_id'},axis=1)
nodes['node_label']	= np.nan
nodes['node_definition'] = np.nan
nodes['node_synonyms'] = np.nan
nodes['node_dbxrefs']= np.nan
nodes['node_namespace'] = np.nan


In [45]:
# /home/stearb/U24/data/MTP/
nodes.to_csv('/mnt/isilon/opentargets/U24KG/data/owlnets_nodes_and_edges/MTPEXP/OWLNETS_node_metadata.txt',
             sep= "\t",index=False)

In [10]:
pd.read_csv('/home/stearb/U24/data/MTP/OWLNETS_node_metadata.txt',sep='\t')

,node_id,node_label,node_definition,node_synonyms,node_dbxrefs,node_namespace
0,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...,NaN,NaN,NaN,NaN,NaN
1,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000002...,NaN,NaN,NaN,NaN,NaN
2,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...,NaN,NaN,NaN,NaN,NaN
3,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000002...,NaN,NaN,NaN,NaN,NaN
4,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1641202,MTPEXP:Wilms_tumor_ENSG00000203995,NaN,NaN,NaN,NaN,NaN
1641203,MTPEXP:Wilms_tumor_ENSG00000162378,NaN,NaN,NaN,NaN,NaN
1641204,MTPEXP:Wilms_tumor_ENSG00000159840,NaN,NaN,NaN,NaN,NaN
1641205,MTPEXP:Wilms_tumor_ENSG00000074755,NaN,NaN,NaN,NaN,NaN


In [12]:
pd.read_csv('/home/stearb/U24/data/MTP/OWLNETS_edgelist.txt',sep='\t')

,subject,predicate,object
0,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...,disease,MONDO:0004967
1,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000002...,disease,MONDO:0004967
2,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...,disease,MONDO:0004967
3,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000002...,disease,MONDO:0004967
4,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...,disease,MONDO:0004967
...,...,...,...
4923616,MTPEXP:Wilms_tumor_ENSG00000203995,tumor_expression,EXPBINS:0.2.0.3
4923617,MTPEXP:Wilms_tumor_ENSG00000162378,tumor_expression,EXPBINS:11.12
4923618,MTPEXP:Wilms_tumor_ENSG00000159840,tumor_expression,EXPBINS:100.200
4923619,MTPEXP:Wilms_tumor_ENSG00000074755,tumor_expression,EXPBINS:6.7


In [37]:
nodes.drop_duplicates()

,node_id
0,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...
1,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000002...
2,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...
3,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000002...
4,MTPEXP:Acute_Lymphoblastic_Leukemia_ENSG000001...
...,...
1641202,MTPEXP:Wilms_tumor_ENSG00000203995
1641203,MTPEXP:Wilms_tumor_ENSG00000162378
1641204,MTPEXP:Wilms_tumor_ENSG00000159840
1641205,MTPEXP:Wilms_tumor_ENSG00000074755


In [18]:
#df.filter(pl.col('Gene_symbol') == 'A1BG')

datasourceId,targetFromSourceId,Gene_symbol,GTEx_subgroup,MONDO,comparisonId,Disease_MeanTpm,log2FoldChange,padj
str,str,str,str,str,str,f64,f64,f64
"""all_cohorts_vs_GTEx""","""ENSG00000121410""","""A1BG""","""Adipose - Subcutaneous""","""MONDO_0004967""","""all_cohorts_Acute_Lymphoblastic_Leukemia_v_Adipose_-_Subcutaneous""",21.3,1.936738,4.2835e-230
"""all_cohorts_vs_GTEx""","""ENSG00000121410""","""A1BG""","""Adipose - Subcutaneous""","""MONDO_0018874""","""all_cohorts_Acute_Myeloid_Leukemia_v_Adipose_-_Subcutaneous""",29.99,0.780813,2.8843e-12
"""all_cohorts_vs_GTEx""","""ENSG00000121410""","""A1BG""","""Adipose - Subcutaneous""","""MONDO_0002787""","""all_cohorts_Adamantinomatous_Craniopharyngioma_v_Adipose_-_Subcutaneous""",9.87,0.02919,0.925092
"""all_cohorts_vs_GTEx""","""ENSG00000121410""","""A1BG""","""Adipose - Subcutaneous""","""MONDO_0002684""","""all_cohorts_Atypical_choroid_plexus_papilloma_v_Adipose_-_Subcutaneous""",17.78,-0.315804,0.90536
"""all_cohorts_vs_GTEx""","""ENSG00000121410""","""A1BG""","""Adipose - Subcutaneous""","""MONDO_0020560""","""all_cohorts_Atypical_Teratoid_Rhabdoid_Tumor_v_Adipose_-_Subcutaneous""",7.21,0.019559,0.94685
…,…,…,…,…,…,…,…,…
"""all_cohorts_vs_GTEx""","""ENSG00000121410""","""A1BG""","""Adipose - Subcutaneous""","""MONDO_0005089""","""all_cohorts_Sarcoma_v_Adipose_-_Subcutaneous""",4.17,-0.152431,0.999985
"""all_cohorts_vs_GTEx""","""ENSG00000121410""","""A1BG""","""Adipose - Subcutaneous""","""MONDO_0002546""","""all_cohorts_Schwannoma_v_Adipose_-_Subcutaneous""",20.57,0.073285,0.890659
"""all_cohorts_vs_GTEx""","""ENSG00000121410""","""A1BG""","""Adipose - Subcutaneous""","""MONDO_0016693""","""all_cohorts_Subependymal_Giant_Cell_Astrocytoma_v_Adipose_-_Subcutaneous""",8.27,-0.064077,0.999981


In [7]:
#df = nbl_gc_all.filter(pl.col('datasourceId')=='all_cohorts_vs_GTEx')\
#                        .unique('targetFromSourceId')\
#                        .select(['datasourceId','targetFromSourceId','MONDO','comparisonId','Disease_MeanTpm'])
#np.shape(df)

(54274, 5)

In [8]:

q = pl.scan_parquet(path+pfile).select('datasourceId').unique() #.filter(pl.col('datasourceId')=='TARGET_vs_GTEx')
datasources = q.collect().to_series().to_numpy()
datasources

array(['GMKF_vs_GTEx', 'all_cohorts_vs_GTEx', 'TARGET_vs_GTEx',
       'PBTA_vs_GTEx'], dtype=object)

In [12]:
q=pl.scan_parquet(path+pfile).filter(pl.col('datasourceId')=='GMKF_vs_GTEx').\
                    filter(pl.col('targetFromSourceId')=='ENSG00000121410').\
                    filter(pl.col('GTEx_subgroup')=='Adrenal Gland')

q.collect()

datasourceId,datatypeId,Dataset,Gene_symbol,targetFromSourceId,PMTL,diseaseFromSourceMappedId,MONDO,GTEx_tissue_subgroup_UBERON,comparisonId,Disease,Disease_Count,GTEx_subgroup,GTEx_Count,Disease_MeanTpm,GTEx_MeanTpm,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64
"""GMKF_vs_GTEx""","""rna_expression""","""GMKF""","""A1BG""","""ENSG00000121410""",null,"""EFO_0000502""","""MONDO_0005035""","""UBERON_0002369""","""GMKF_Ganglioneuroblastoma_v_Adrenal_Gland""","""Ganglioneuroblastoma""",25,"""Adrenal Gland""",258,7.27,1.59,135.100783,0.095815,0.166378,0.575888,0.564691,0.805859
"""GMKF_vs_GTEx""","""rna_expression""","""GMKF""","""A1BG""","""ENSG00000121410""",null,"""EFO_0000621""","""MONDO_0005072""","""UBERON_0002369""","""GMKF_Neuroblastoma_v_Adrenal_Gland""","""Neuroblastoma""",183,"""Adrenal Gland""",258,8.13,1.59,150.304843,1.201062,0.076061,15.790759,3.6022e-56,2.2392e-55


In [11]:
q=pl.scan_parquet(path+pfile).filter(pl.col('datasourceId')=='TARGET_vs_GTEx').\
                    filter(pl.col('targetFromSourceId')=='ENSG00000121410').\
                    filter(pl.col('GTEx_subgroup')=='Adrenal Gland')

#.select('Disease').unique()
q.collect()

datasourceId,datatypeId,Dataset,Gene_symbol,targetFromSourceId,PMTL,diseaseFromSourceMappedId,MONDO,GTEx_tissue_subgroup_UBERON,comparisonId,Disease,Disease_Count,GTEx_subgroup,GTEx_Count,Disease_MeanTpm,GTEx_MeanTpm,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64
"""TARGET_vs_GTEx""","""rna_expression""","""TARGET""","""A1BG""","""ENSG00000121410""",null,"""EFO_0000220""","""MONDO_0004967""","""UBERON_0002369""","""TARGET_Acute_Lymphoblastic_Leukemia_v_Adrenal_Gland""","""Acute Lymphoblastic Leukemia""",538,"""Adrenal Gland""",258,21.3,1.59,377.942435,0.237516,0.110588,2.147746,0.031734,0.05238
"""TARGET_vs_GTEx""","""rna_expression""","""TARGET""","""A1BG""","""ENSG00000121410""",null,"""EFO_0000222""","""MONDO_0018874""","""UBERON_0002369""","""TARGET_Acute_Myeloid_Leukemia_v_Adrenal_Gland""","""Acute Myeloid Leukemia""",168,"""Adrenal Gland""",258,29.99,1.59,411.143181,2.46361,0.133375,18.471237,3.5193e-76,1.2221e-74
"""TARGET_vs_GTEx""","""rna_expression""","""TARGET""","""A1BG""","""ENSG00000121410""",null,"""EFO_0000350""","""MONDO_0005006""","""UBERON_0002369""","""TARGET_Clear_cell_sarcoma_of_the_kidney_v_Adrenal_Gland""","""Clear cell sarcoma of the kidney""",13,"""Adrenal Gland""",258,0.21,1.59,116.542623,-0.218484,0.23346,-0.935851,0.34935,0.868793
"""TARGET_vs_GTEx""","""rna_expression""","""TARGET""","""A1BG""","""ENSG00000121410""",null,"""EFO_0000502""","""MONDO_0005035""","""UBERON_0002369""","""TARGET_Ganglioneuroblastoma_v_Adrenal_Gland""","""Ganglioneuroblastoma""",20,"""Adrenal Gland""",258,35.94,1.59,159.172623,0.221326,0.222058,0.996705,0.318908,0.699852
"""TARGET_vs_GTEx""","""rna_expression""","""TARGET""","""A1BG""","""ENSG00000121410""",null,"""EFO_0000621""","""MONDO_0005072""","""UBERON_0002369""","""TARGET_Neuroblastoma_v_Adrenal_Gland""","""Neuroblastoma""",129,"""Adrenal Gland""",258,34.37,1.59,285.657011,1.467744,0.120166,12.214265,2.6085e-34,3.7140e-33
"""TARGET_vs_GTEx""","""rna_expression""","""TARGET""","""A1BG""","""ENSG00000121410""",null,"""EFO_0000637""","""MONDO_0009807""","""UBERON_0002369""","""TARGET_Osteosarcoma_v_Adrenal_Gland""","""Osteosarcoma""",87,"""Adrenal Gland""",258,36.2,1.59,185.130903,1.024455,0.150611,6.801987,1.0319e-11,1.1211e-10
"""TARGET_vs_GTEx""","""rna_expression""","""TARGET""","""A1BG""","""ENSG00000121410""",null,"""EFO_1000512""","""MONDO_0002729""","""UBERON_0002369""","""TARGET_Rhabdoid_tumor_of_the_kidney_v_Adrenal_Gland""","""Rhabdoid tumor of the kidney""",64,"""Adrenal Gland""",258,44.59,1.59,311.691068,1.399992,0.193708,7.227325,4.9260e-13,1.6332e-11
"""TARGET_vs_GTEx""","""rna_expression""","""TARGET""","""A1BG""","""ENSG00000121410""",null,"""MONDO_0006058""","""MONDO_0006058""","""UBERON_0002369""","""TARGET_Wilms_tumor_v_Adrenal_Gland""","""Wilms tumor""",124,"""Adrenal Gland""",258,16.65,1.59,176.863317,0.506462,0.118221,4.284039,0.000018,0.000041


In [7]:
for DATASOURCE in datasources:
    print(DATASOURCE,':\n')
    q=pl.scan_parquet(path+pfile).filter(pl.col('datasourceId')==DATASOURCE).select('Disease').unique()
    print('\t',q.collect())

PBTA_vs_GTEx :

	 shape: (43, 1)
┌──────────────────────────────────────────────────────┐
│ Disease                                              │
│ ---                                                  │
│ str                                                  │
╞══════════════════════════════════════════════════════╡
│ Neurofibroma/Plexiform                               │
│ Ependymoma                                           │
│ Adamantinomatous Craniopharyngioma                   │
│ CNS Embryonal tumor                                  │
│ …                                                    │
│ Low-grade glioma                                     │
│ Neuroblastoma                                        │
│ Desmoplastic infantile astrocytoma and ganglioglioma │
│ Atypical choroid plexus papilloma                    │
└──────────────────────────────────────────────────────┘
GMKF_vs_GTEx :

	 shape: (2, 1)
┌──────────────────────┐
│ Disease              │
│ ---                  │
│ str

In [12]:
df.dtypes

[String, String, String, String, Float64]

datasourceId,targetFromSourceId,MONDO,comparisonId,Disease_MeanTpm,bins
str,str,str,str,f64,cat
"""all_cohorts_vs_GTEx""","""ENSG00000266840""","""MONDO_0004967""","""all_cohorts_Acute_Lymphoblastic_Leukemia_v_Adipose_-_Subcutaneous""",0.0,"""(-inf, 0]"""
"""all_cohorts_vs_GTEx""","""ENSG00000115297""","""MONDO_0004967""","""all_cohorts_Acute_Lymphoblastic_Leukemia_v_Adipose_-_Subcutaneous""",0.39,"""(0.3, 0.4]"""
"""all_cohorts_vs_GTEx""","""ENSG00000255170""","""MONDO_0004967""","""all_cohorts_Acute_Lymphoblastic_Leukemia_v_Adipose_-_Subcutaneous""",0.01,"""(0.009, 0.01]"""
"""all_cohorts_vs_GTEx""","""ENSG00000253369""","""MONDO_0004967""","""all_cohorts_Acute_Lymphoblastic_Leukemia_v_Adipose_-_Subcutaneous""",0.05,"""(0.04, 0.05]"""
"""all_cohorts_vs_GTEx""","""ENSG00000280229""","""MONDO_0004967""","""all_cohorts_Acute_Lymphoblastic_Leukemia_v_Adipose_-_Subcutaneous""",0.0,"""(-inf, 0]"""
…,…,…,…,…,…
"""all_cohorts_vs_GTEx""","""ENSG00000185903""","""MONDO_0004967""","""all_cohorts_Acute_Lymphoblastic_Leukemia_v_Adipose_-_Subcutaneous""",0.0,"""(-inf, 0]"""
"""all_cohorts_vs_GTEx""","""ENSG00000112541""","""MONDO_0004967""","""all_cohorts_Acute_Lymphoblastic_Leukemia_v_Adipose_-_Subcutaneous""",0.78,"""(0.7, 0.8]"""
"""all_cohorts_vs_GTEx""","""ENSG00000124116""","""MONDO_0004967""","""all_cohorts_Acute_Lymphoblastic_Leukemia_v_Adipose_-_Subcutaneous""",0.08,"""(0.07, 0.08]"""


In [25]:
#pl.all(df.select('Disease_MeanTpm') == df_binned.select('Disease_MeanTpm'))

In [13]:
nbl_gc_all.filter((pl.col('Gene_symbol') == 'A1BG')  & (pl.col('GTEx_tissue_subgroup_UBERON') == 'UBERON_0002190') )


datasourceId,datatypeId,Dataset,Gene_symbol,targetFromSourceId,PMTL,diseaseFromSourceMappedId,MONDO,GTEx_tissue_subgroup_UBERON,comparisonId,Disease,Disease_Count,GTEx_subgroup,GTEx_Count,Disease_MeanTpm,GTEx_MeanTpm,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64
"""all_cohorts_vs_GTEx""","""rna_expression""","""All Cohorts""","""A1BG""","""ENSG00000121410""",null,"""EFO_0000220""","""MONDO_0004967""","""UBERON_0002190""","""all_cohorts_Acute_Lymphoblastic_Leukemia_v_Adipose_-_Subcutaneous""","""Acute Lymphoblastic Leukemia""",538,"""Adipose - Subcutaneous""",663,21.3,3.32,369.041594,1.936738,0.059641,32.473496,2.5245e-231,4.2835e-230
"""TARGET_vs_GTEx""","""rna_expression""","""TARGET""","""A1BG""","""ENSG00000121410""",null,"""EFO_0000220""","""MONDO_0004967""","""UBERON_0002190""","""TARGET_Acute_Lymphoblastic_Leukemia_v_Adipose_-_Subcutaneous""","""Acute Lymphoblastic Leukemia""",538,"""Adipose - Subcutaneous""",663,21.3,3.32,369.041594,1.936738,0.059641,32.473496,2.5245e-231,4.2835e-230


In [10]:
nbl_gc_all.select('datasourceId').unique()

datasourceId
str
"""TARGET_vs_GTEx"""
"""all_cohorts_vs_GTEx"""


In [6]:
df = nbl_gc_all.to_pandas()


In [7]:
df[df['Gene_symbol'] == 'A1BG']

,datasourceId,datatypeId,Dataset,Gene_symbol,targetFromSourceId,PMTL,diseaseFromSourceMappedId,MONDO,GTEx_tissue_subgroup_UBERON,comparisonId,Disease,Disease_Count,GTEx_subgroup,GTEx_Count,Disease_MeanTpm,GTEx_MeanTpm,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,all_cohorts_vs_GTEx,rna_expression,All Cohorts,A1BG,ENSG00000121410,None,EFO_0000621,MONDO_0005072,UBERON_0002190,all_cohorts_Neuroblastoma_v_Adipose_-_Subcutan...,Neuroblastoma,304,Adipose - Subcutaneous,663,19.19,3.32,256.999942,0.555308,0.062144,8.935772,4.043460e-19,1.713718e-18
54274,all_cohorts_vs_GTEx,rna_expression,All Cohorts,A1BG,ENSG00000121410,None,EFO_0000621,MONDO_0005072,UBERON_0010414,all_cohorts_Neuroblastoma_v_Adipose_-_Visceral...,Neuroblastoma,304,Adipose - Visceral (Omentum),541,19.19,3.90,280.763008,0.469265,0.061784,7.595207,3.073004e-14,9.326241e-14
108548,all_cohorts_vs_GTEx,rna_expression,All Cohorts,A1BG,ENSG00000121410,None,EFO_0000621,MONDO_0005072,UBERON_0002369,all_cohorts_Neuroblastoma_v_Adrenal_Gland,Neuroblastoma,304,Adrenal Gland,258,19.19,1.59,228.649821,1.393243,0.075836,18.371774,2.210667e-75,2.397381e-74
162822,all_cohorts_vs_GTEx,rna_expression,All Cohorts,A1BG,ENSG00000121410,None,EFO_0000621,MONDO_0005072,UBERON_0001496,all_cohorts_Neuroblastoma_v_Artery_-_Aorta,Neuroblastoma,304,Artery - Aorta,432,19.19,3.72,253.780729,0.919974,0.066656,13.801818,2.485056e-43,1.194482e-42
217096,all_cohorts_vs_GTEx,rna_expression,All Cohorts,A1BG,ENSG00000121410,None,EFO_0000621,MONDO_0005072,UBERON_0001621,all_cohorts_Neuroblastoma_v_Artery_-_Coronary,Neuroblastoma,304,Artery - Coronary,240,19.19,7.80,344.381670,0.028600,0.065501,0.436631,6.623788e-01,8.758138e-01
271370,all_cohorts_vs_GTEx,rna_expression,All Cohorts,A1BG,ENSG00000121410,None,EFO_0000621,MONDO_0005072,UBERON_0007610,all_cohorts_Neuroblastoma_v_Artery_-_Tibial,Neuroblastoma,304,Artery - Tibial,663,19.19,6.73,345.964225,-0.043319,0.053853,-0.804406,4.211623e-01,4.982227e-01
325644,all_cohorts_vs_GTEx,rna_expression,All Cohorts,A1BG,ENSG00000121410,None,EFO_0000621,MONDO_0005072,UBERON_0001255,all_cohorts_Neuroblastoma_v_Bladder,Neuroblastoma,304,Bladder,21,19.19,2.11,275.593394,0.029175,0.202772,0.143879,8.855959e-01,9.207145e-01
379918,all_cohorts_vs_GTEx,rna_expression,All Cohorts,A1BG,ENSG00000121410,None,EFO_0000621,MONDO_0005072,UBERON_0013756,all_cohorts_Neuroblastoma_v_Blood,Neuroblastoma,304,Blood,755,19.19,1.99,173.714360,0.298783,0.055793,5.355201,8.546117e-08,1.824514e-07
434192,all_cohorts_vs_GTEx,rna_expression,All Cohorts,A1BG,ENSG00000121410,None,EFO_0000621,MONDO_0005072,UBERON_0001876,all_cohorts_Neuroblastoma_v_Brain_-_Amygdala,Neuroblastoma,304,Brain - Amygdala,152,19.19,4.57,314.059325,-0.075410,0.072098,-1.045932,2.955923e-01,3.404928e-01
488466,all_cohorts_vs_GTEx,rna_expression,All Cohorts,A1BG,ENSG00000121410,None,EFO_0000621,MONDO_0005072,UBERON_0009835,all_cohorts_Neuroblastoma_v_Brain_-_Anterior_c...,Neuroblastoma,304,Brain - Anterior cingulate cortex (BA24),176,19.19,3.54,286.953582,0.212454,0.059750,3.555723,3.769412e-04,7.284535e-04


In [26]:
nbl_gc_all['MTPEXP_CODEID'] = 'MTPEXP ' + nbl_gc_all['Disease'] + '_' + \
                                nbl_gc_all['Gene_symbol'] + '_' + nbl_gc_all['GTEx_subgroup']

nbl_gc_all

TypeError: DataFrame object does not support `Series` assignment by index

Use `DataFrame.with_columns`.

In [16]:
df = nbl_gc_all.to_pandas()

df[['targetFromSourceId','MONDO', 'GTEx_tissue_subgroup_UBERON','log2FoldChange']]

,targetFromSourceId,MONDO,GTEx_tissue_subgroup_UBERON,log2FoldChange
0,ENSG00000121410,MONDO_0005072,UBERON_0002190,0.555308
1,ENSG00000268895,MONDO_0005072,UBERON_0002190,2.208708
2,ENSG00000148584,MONDO_0005072,UBERON_0002190,0.792896
3,ENSG00000175899,MONDO_0005072,UBERON_0002190,-0.713397
4,ENSG00000245105,MONDO_0005072,UBERON_0002190,0.153147
...,...,...,...,...
2876517,ENSG00000162378,MONDO_0005072,UBERON_0000996,-0.325999
2876518,ENSG00000159840,MONDO_0005072,UBERON_0000996,-0.787445
2876519,ENSG00000274572,MONDO_0005072,UBERON_0000996,NaN
2876520,ENSG00000074755,MONDO_0005072,UBERON_0000996,-0.524602


# Check Daves files

In [8]:
edges = pd.read_csv('/home/stearb/U24/data/gene-counts-rsem_filtered_gene_to_disease_edge.csv')
edges

,subject,predicate,object
1,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000121410
2,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000268895
3,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000148584
4,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000175899
5,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000245105
...,...,...,...
5753040,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000162378
5753041,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000159840
5753042,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000274572
5753043,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000074755


In [15]:
# find uniq data sources in daves file first and filter by them and thendo the binning
# KF and ALL...

for col in edges.itertuples():
    mondo_code = col[1].replace(' ','_')
    gene_code = col[3].split(' ')[-1]
    
    q=pl.scan_parquet(path+pfile).filter(pl.col('datasourceId')=='TARGET_vs_GTEx').\
                    filter(pl.col('targetFromSourceId')=='ENSG00000121410').\
                    filter(pl.col('GTEx_subgroup')=='Adrenal Gland')
    break

In [17]:
gene_code

'ENSG00000121410'

In [7]:
edges[edges['subject'].str.startswith('MONDO')]

,subject,predicate,object


In [5]:
nodes = pd.read_csv('/home/stearb/U24/data/gene-counts-rsem_filtered_gene_to_disease_node.csv')
nodes

,MONDO,predicate,targetFromSourceId
1,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000232320
2,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000196604
3,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000280081
4,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000248156
5,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000227050
...,...,...,...
54270,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000266533
54271,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000279382
54272,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000258905
54273,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000265924


In [3]:
import pandas as pd
edges = pd.read_csv('/home/stearb/U24/data/gene-counts-rsem_filtered_gene_to_disease_edge.csv')
edges

,subject,predicate,object
1,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000121410
2,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000268895
3,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000148584
4,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000175899
5,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000245105
...,...,...,...
5753040,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000162378
5753041,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000159840
5753042,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000274572
5753043,MONDO 0004967,associated with,KF_ALL_MTP_EXP_ENSG ENSG00000074755


In [1]:
!ls /home/stearb/U24/data/

flatpak: symbol lookup error: /lib64/librpmio.so.9: undefined symbol: lzma_stream_encoder_mt, version XZ_5.2
gene-counts-rsem_filtered_gene_to_disease_edge.csv  helper_files
gene-counts-rsem_filtered_gene_to_disease_node.csv  MTP


In [8]:
nbl_gc_all = drop_single_unique_value_cols(nbl_gc_all)

Dropping "datasourceId" column, the only value is: ['all_cohorts_vs_GTEx'].
Dropping "datatypeId" column, the only value is: ['rna_expression'].
Dropping "Dataset" column, the only value is: ['All Cohorts'].
Dropping "diseaseFromSourceMappedId" column, the only value is: ['EFO_0000621'].
Dropping "MONDO" column, the only value is: ['MONDO_0005072'].
Dropping "Disease" column, the only value is: ['Neuroblastoma'].
Dropping "Disease_Count" column, the only value is: [304].


In [10]:
# see # of unique genes for each tissue. its 54,274
for gtex_subgroup in nbl_gc_all.select("GTEx_tissue_subgroup_UBERON").unique().to_numpy():
        num_unique_genes = len( nbl_gc_all.filter(
                                            pl.col('GTEx_tissue_subgroup_UBERON') == gtex_subgroup[0])\
                                            .select("Gene_symbol").unique() )
        print(f'# unique genes for {gtex_subgroup}: {num_unique_genes}')


# unique genes for ['UBERON_0006330']: 54274
# unique genes for ['UBERON_0000458']: 54274
# unique genes for ['UBERON_0001882']: 54274
# unique genes for ['UBERON_0006631']: 54274
# unique genes for ['UBERON_0000007']: 54274
# unique genes for ['UBERON_0006469']: 54274
# unique genes for ['UBERON_0008952']: 54274
# unique genes for ['UBERON_0002038']: 54274
# unique genes for ['UBERON_0000945']: 54274
# unique genes for ['UBERON_0001114']: 54274
# unique genes for ['UBERON_0036149']: 54274
# unique genes for ['UBERON_0001621']: 54274
# unique genes for ['UBERON_0007610']: 54274
# unique genes for ['UBERON_0000995']: 54274
# unique genes for ['UBERON_0004550']: 54274
# unique genes for ['UBERON_0000996']: 54274
# unique genes for ['UBERON_0001150']: 54274
# unique genes for ['UBERON_0001293']: 54274
# unique genes for ['UBERON_0012249']: 54274
# unique genes for ['UBERON_0002046']: 54274
# unique genes for ['UBERON_0001870']: 54274
# unique genes for ['UBERON_0003889']: 54274
# unique g

In [23]:
nbl_gc_df = nbl_gc_all.to_pandas()
len(nbl_gc_df)

2876522

In [25]:
nbl_gc_df[nbl_gc_df['Gene_symbol'] == "H2AC17"]#.shape #.unique()

,Gene_symbol,targetFromSourceId,PMTL,GTEx_tissue_subgroup_UBERON,comparisonId,GTEx_subgroup,GTEx_Count,Disease_MeanTpm,GTEx_MeanTpm,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
25761,H2AC17,ENSG00000278677,None,UBERON_0002190,all_cohorts_Neuroblastoma_v_Adipose_-_Subcutan...,Adipose - Subcutaneous,663,80.8,0.00,384.320138,30.000000,0.414148,72.437846,0.000000e+00,0.000000e+00
80035,H2AC17,ENSG00000278677,None,UBERON_0010414,all_cohorts_Neuroblastoma_v_Adipose_-_Visceral...,Adipose - Visceral (Omentum),541,80.8,0.00,408.552019,17.756696,0.370668,47.904558,0.000000e+00,0.000000e+00
134309,H2AC17,ENSG00000278677,None,UBERON_0002369,all_cohorts_Neuroblastoma_v_Adrenal_Gland,Adrenal Gland,258,80.8,0.00,568.989850,1.700548,0.396749,4.286207,1.817493e-05,3.414641e-05
188583,H2AC17,ENSG00000278677,None,UBERON_0001496,all_cohorts_Neuroblastoma_v_Artery_-_Aorta,Artery - Aorta,432,80.8,0.00,446.521301,30.000000,0.323714,92.674282,0.000000e+00,0.000000e+00
242857,H2AC17,ENSG00000278677,None,UBERON_0001621,all_cohorts_Neuroblastoma_v_Artery_-_Coronary,Artery - Coronary,240,80.8,0.00,587.774476,1.081721,0.399265,2.709281,6.742917e-03,1.170370e-02
297131,H2AC17,ENSG00000278677,None,UBERON_0007610,all_cohorts_Neuroblastoma_v_Artery_-_Tibial,Artery - Tibial,663,80.8,0.00,357.785353,17.463075,0.411288,42.459516,0.000000e+00,0.000000e+00
351405,H2AC17,ENSG00000278677,None,UBERON_0001255,all_cohorts_Neuroblastoma_v_Bladder,Bladder,21,80.8,0.00,838.229299,-6.970014,0.588744,-11.838783,2.459953e-32,1.208149e-29
405679,H2AC17,ENSG00000278677,None,UBERON_0013756,all_cohorts_Neuroblastoma_v_Blood,Blood,755,80.8,0.00,175.802710,16.093714,0.420606,38.263132,0.000000e+00,0.000000e+00
459953,H2AC17,ENSG00000278677,None,UBERON_0001876,all_cohorts_Neuroblastoma_v_Brain_-_Amygdala,Brain - Amygdala,152,80.8,0.00,591.283079,-1.601485,0.383167,-4.179601,2.920210e-05,5.764705e-05
514227,H2AC17,ENSG00000278677,None,UBERON_0009835,all_cohorts_Neuroblastoma_v_Brain_-_Anterior_c...,Brain - Anterior cingulate cortex (BA24),176,80.8,0.00,598.515499,-0.640563,0.391548,-1.635977,1.018444e-01,1.379117e-01


In [26]:
nbl_gc_df.isnull().sum()

Gene_symbol                          0
targetFromSourceId                   0
PMTL                           2851241
GTEx_tissue_subgroup_UBERON     108548
comparisonId                         0
GTEx_subgroup                        0
GTEx_Count                           0
Disease_MeanTpm                      0
GTEx_MeanTpm                         0
baseMean                             0
log2FoldChange                  106256
lfcSE                           106256
stat                            106256
pvalue                          106632
padj                            497409
dtype: int64

# Select rows where log2FC is > 5 and adj-Pvalue is less than .05


In [36]:

nbl_gc_filt = nbl_gc_df[(nbl_gc_df['log2FoldChange'] > 5) & \
                        (nbl_gc_df['padj'] < .05)] #.sort_values('log2FoldChange', ascending=False)
nbl_gc_filt

,Gene_symbol,targetFromSourceId,PMTL,GTEx_tissue_subgroup_UBERON,comparisonId,GTEx_subgroup,GTEx_Count,Disease_MeanTpm,GTEx_MeanTpm,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
15,AACSP1,ENSG00000250420,None,UBERON_0002190,all_cohorts_Neuroblastoma_v_Adipose_-_Subcutan...,Adipose - Subcutaneous,663,20.73,0.06,652.541094,8.204093,0.241093,34.028815,8.353002e-254,1.260532e-251
265,ACTBP7,ENSG00000185607,None,UBERON_0002190,all_cohorts_Neuroblastoma_v_Adipose_-_Subcutan...,Adipose - Subcutaneous,663,2.05,0.03,29.828564,5.778283,0.234006,24.692878,1.275525e-134,4.335283e-133
295,ACTL6B,ENSG00000077080,Relevant Molecular Target,UBERON_0002190,all_cohorts_Neuroblastoma_v_Adipose_-_Subcutan...,Adipose - Subcutaneous,663,55.42,0.04,742.944688,9.531917,0.263560,36.166003,2.084988e-286,4.712361e-284
395,ADAMTS20,ENSG00000173157,None,UBERON_0002190,all_cohorts_Neuroblastoma_v_Adipose_-_Subcutan...,Adipose - Subcutaneous,663,0.18,0.00,8.711918,5.507206,0.331060,16.635082,3.882259e-62,4.451844e-61
437,ADCY8,ENSG00000155897,None,UBERON_0002190,all_cohorts_Neuroblastoma_v_Adipose_-_Subcutan...,Adipose - Subcutaneous,663,2.38,0.10,135.771373,5.076232,0.248349,20.439933,7.383977e-93,1.406943e-91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875312,XAGE1A,ENSG00000204379,None,UBERON_0000996,all_cohorts_Neuroblastoma_v_Vagina,Vagina,156,10.37,0.00,55.009291,5.405979,0.733918,7.365921,1.759279e-13,8.023563e-13
2875313,XAGE1B,ENSG00000204382,None,UBERON_0000996,all_cohorts_Neuroblastoma_v_Vagina,Vagina,156,27.41,0.00,143.441869,5.771607,0.588797,9.802379,1.099644e-22,8.059674e-22
2875490,ZACN,ENSG00000186919,None,UBERON_0000996,all_cohorts_Neuroblastoma_v_Vagina,Vagina,156,24.06,0.18,987.522207,7.713143,0.171876,44.876215,0.000000e+00,0.000000e+00
2875551,ZBTB45P1,ENSG00000225108,None,UBERON_0000996,all_cohorts_Neuroblastoma_v_Vagina,Vagina,156,0.23,0.01,5.152832,5.115554,0.336772,15.189984,4.120229e-52,1.024101e-50


In [39]:
uniq_genes = nbl_gc_filt['Gene_symbol'].unique()
print(len(uniq_genes))

4712


In [42]:
nbl_gc_df[nbl_gc_df['Gene_symbol'].isin(uniq_genes)]

,Gene_symbol,targetFromSourceId,PMTL,GTEx_tissue_subgroup_UBERON,comparisonId,GTEx_subgroup,GTEx_Count,Disease_MeanTpm,GTEx_MeanTpm,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
15,AACSP1,ENSG00000250420,None,UBERON_0002190,all_cohorts_Neuroblastoma_v_Adipose_-_Subcutan...,Adipose - Subcutaneous,663,20.73,0.06,652.541094,8.204093,0.241093,34.028815,8.353002e-254,1.260532e-251
40,AATK,ENSG00000181409,None,UBERON_0002190,all_cohorts_Neuroblastoma_v_Adipose_-_Subcutan...,Adipose - Subcutaneous,663,22.26,7.97,1921.834781,1.176180,0.101195,11.622930,3.151414e-31,1.973456e-30
46,ABCA12,ENSG00000144452,None,UBERON_0002190,all_cohorts_Neuroblastoma_v_Adipose_-_Subcutan...,Adipose - Subcutaneous,663,1.03,0.09,104.266580,4.970075,0.200444,24.795328,1.006809e-135,3.472501e-134
50,ABCA3,ENSG00000167972,None,UBERON_0002190,all_cohorts_Neuroblastoma_v_Adipose_-_Subcutan...,Adipose - Subcutaneous,663,33.63,11.73,3541.119924,1.450231,0.067146,21.598040,1.873978e-103,4.160673e-102
58,ABCB1,ENSG00000085563,None,UBERON_0002190,all_cohorts_Neuroblastoma_v_Adipose_-_Subcutan...,Adipose - Subcutaneous,663,14.42,3.38,769.565575,1.273908,0.094787,13.439650,3.541076e-41,2.731982e-40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2876431,ZNF99,ENSG00000213973,None,UBERON_0000996,all_cohorts_Neuroblastoma_v_Vagina,Vagina,156,0.43,0.06,85.455861,-0.901276,0.231987,-3.885034,1.023156e-04,2.398244e-04
2876449,ZNRF3-IT1,ENSG00000235786,None,UBERON_0000996,all_cohorts_Neuroblastoma_v_Vagina,Vagina,156,0.86,0.04,11.130927,-1.379826,0.319714,-4.315810,1.590189e-05,4.048878e-05
2876466,ZRANB2-AS2,ENSG00000229956,None,UBERON_0000996,all_cohorts_Neuroblastoma_v_Vagina,Vagina,156,2.27,0.10,85.537340,-0.572459,0.168654,-3.394277,6.881005e-04,1.481566e-03
2876470,ZSCAN1,ENSG00000152467,None,UBERON_0000996,all_cohorts_Neuroblastoma_v_Vagina,Vagina,156,5.97,0.31,86.578573,1.889183,0.135918,13.899397,6.387726e-44,1.155892e-42


In [30]:
nbl_gc_all.sort('log2FoldChange',descending=True)

Gene_symbol,targetFromSourceId,PMTL,GTEx_tissue_subgroup_UBERON,comparisonId,GTEx_subgroup,GTEx_Count,Disease_MeanTpm,GTEx_MeanTpm,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
str,str,str,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64
"""ENSG00000270066""","""ENSG00000270066""",null,"""UBERON_0002190""","""all_cohorts_Neuroblastoma_v_Adipose_-_Subcutaneous""","""Adipose - Subcutaneous""",663,55.62,0.0,224.177944,30.0,0.526805,56.947047,0.0,0.0
"""H2AC17""","""ENSG00000278677""",null,"""UBERON_0002190""","""all_cohorts_Neuroblastoma_v_Adipose_-_Subcutaneous""","""Adipose - Subcutaneous""",663,80.8,0.0,384.320138,30.0,0.414148,72.437846,0.0,0.0
"""RNU1-3""","""ENSG00000207513""",null,"""UBERON_0002190""","""all_cohorts_Neuroblastoma_v_Adipose_-_Subcutaneous""","""Adipose - Subcutaneous""",663,1164.38,0.0,339.112025,30.0,0.423844,70.780745,0.0,0.0
"""H2AC17""","""ENSG00000278677""",null,"""UBERON_0001496""","""all_cohorts_Neuroblastoma_v_Artery_-_Aorta""","""Artery - Aorta""",432,80.8,0.0,446.521301,30.0,0.323714,92.674282,0.0,0.0
"""ENSG00000270066""","""ENSG00000270066""",null,"""UBERON_0007610""","""all_cohorts_Neuroblastoma_v_Artery_-_Tibial""","""Artery - Tibial""",663,55.62,0.0,208.201446,30.0,0.524117,57.239177,0.0,0.0
"""MIR3648-2""","""ENSG00000264462""",null,"""UBERON_0007610""","""all_cohorts_Neuroblastoma_v_Artery_-_Tibial""","""Artery - Tibial""",663,441.99,0.0,172.896279,30.0,0.5598,53.590551,0.0,0.0
"""ENSG00000270066""","""ENSG00000270066""",null,null,"""all_cohorts_Neuroblastoma_v_Cells_-_Cultured_fibroblasts""","""Cells - Cultured fibroblasts""",504,55.62,0.0,278.718376,30.0,0.463393,64.739871,0.0,0.0
"""H2AC17""","""ENSG00000278677""",null,null,"""all_cohorts_Neuroblastoma_v_Cells_-_Cultured_fibroblasts""","""Cells - Cultured fibroblasts""",504,80.8,0.0,476.039526,30.0,0.356617,84.123938,0.0,0.0
"""ENSG00000270066""","""ENSG00000270066""",null,"""UBERON_0001157""","""all_cohorts_Neuroblastoma_v_Colon_-_Transverse""","""Colon - Transverse""",406,55.62,0.0,276.52326,30.0,0.412334,72.75648,0.0,0.0


In [13]:
nbl_gc_all['PMTL'].unique()

PMTL
str
"""Non-Relevant Molecular Target"""
null
"""Relevant Molecular Target"""


In [20]:
# select random tissue and merge in the HGNC codes to see how many genes
# we lose on the merge for a single tissue
tissue='UBERON_0006330'
nbl_single_tissue = nbl_gc_all.filter(pl.col('GTEx_tissue_subgroup_UBERON') == tissue)
print(nbl_single_tissue.shape)

# Merge
nbl_gc_all_merge = nbl_single_tissue.join(hgnc_select_ens.unique('targetFromSourceId'),on='targetFromSourceId',how='left') 
nbl_single_tissue.head()

hgnc_nulls = nbl_gc_all_merge.filter(pl.col('hgnc_id').is_null())
                                     
print('Lost',hgnc_nulls.shape[0], 'genes for single tissue')

(54274, 15)
Lost 14390 genes for single tissue


In [30]:
hgnc_nulls=hgnc_nulls.select(['Gene_symbol','targetFromSourceId','hgnc_id'])


hgnc_nulls.filter(pl.col('Gene_symbol').str.starts_with('ENS'))

hgnc_nulls

Gene_symbol,targetFromSourceId,hgnc_id
str,str,str
"""AA06""","""ENSG00000265544""",null
"""ANAPC1P2""","""ENSG00000285793""",null
"""ARMCX5-GPRASP2""","""ENSG00000271147""",null
"""BMS1P4""","""ENSG00000271816""",null
"""CBSL""","""ENSG00000274276""",null
"""CH507-42P11.6""","""ENSG00000280179""",null
"""CRAT37""","""ENSG00000258551""",null
"""CRYAA2""","""ENSG00000276076""",null
"""CTC-338M12.4""","""ENSG00000233937""",null


In [35]:
hgnc_nulls.filter(pl.col('Gene_symbol').str.starts_with('ENS'))

Gene_symbol,targetFromSourceId,hgnc_id
str,str,str
"""ENSG00000083622""","""ENSG00000083622""",null
"""ENSG00000103200""","""ENSG00000103200""",null
"""ENSG00000108516""","""ENSG00000108516""",null
"""ENSG00000108958""","""ENSG00000108958""",null
"""ENSG00000121388""","""ENSG00000121388""",null
"""ENSG00000124593""","""ENSG00000124593""",null
"""ENSG00000124835""","""ENSG00000124835""",null
"""ENSG00000131484""","""ENSG00000131484""",null
"""ENSG00000145063""","""ENSG00000145063""",null


In [38]:
'ENSG00000121388' in hgnc_master['ensembl_gene_id']

False

In [34]:
hgnc_nulls['Gene_symbol'].to_numpy()[:100]

array(['AA06', 'ANAPC1P2', 'ARMCX5-GPRASP2', 'BMS1P4', 'CBSL',
       'CH507-42P11.6', 'CRAT37', 'CRYAA2', 'CTC-338M12.4',
       'CTD-2194D22.4', 'CTD-2350J17.1', 'D21S2088E', 'DKFZP434A062',
       'DUXAP10', 'DUXAP8', 'ENSG00000083622', 'ENSG00000103200',
       'ENSG00000108516', 'ENSG00000108958', 'ENSG00000121388',
       'ENSG00000124593', 'ENSG00000124835', 'ENSG00000131484',
       'ENSG00000145063', 'ENSG00000150732', 'ENSG00000165121',
       'ENSG00000166104', 'ENSG00000167046', 'ENSG00000169164',
       'ENSG00000170089', 'ENSG00000170409', 'ENSG00000173357',
       'ENSG00000173862', 'ENSG00000174171', 'ENSG00000175604',
       'ENSG00000176134', 'ENSG00000176183', 'ENSG00000176232',
       'ENSG00000176320', 'ENSG00000176349', 'ENSG00000176515',
       'ENSG00000176882', 'ENSG00000176984', 'ENSG00000177359',
       'ENSG00000177418', 'ENSG00000177452', 'ENSG00000177596',
       'ENSG00000177699', 'ENSG00000177725', 'ENSG00000178107',
       'ENSG00000178715', 'ENSG000001

In [5]:
#nbl_gc_all_merge = nbl_gc_all.join(hgnc_select,on='Gene_symbol') 

# merging w/ ensembl ids instead of gene symbol ids results in about 100,000 more matches 
# but we are still losing about 700k rows

nbl_gc_all_merge = nbl_gc_all.join(hgnc_select_ens.unique('targetFromSourceId'),on='targetFromSourceId',how='left') 
nbl_gc_all_merge.head()

Gene_symbol,targetFromSourceId,PMTL,GTEx_tissue_subgroup_UBERON,comparisonId,GTEx_subgroup,GTEx_Count,Disease_MeanTpm,GTEx_MeanTpm,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,hgnc_id
str,str,str,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""A1BG""","""ENSG00000121410""",null,"""UBERON_0002190""","""all_cohorts_Neuroblastoma_v_Adipose_-_Subcutaneous""","""Adipose - Subcutaneous""",663,19.19,3.32,256.999942,0.555308,0.062144,8.935772,4.0435e-19,1.7137e-18,"""HGNC:5"""
"""A1BG-AS1""","""ENSG00000268895""",null,"""UBERON_0002190""","""all_cohorts_Neuroblastoma_v_Adipose_-_Subcutaneous""","""Adipose - Subcutaneous""",663,6.49,1.71,177.650518,2.208708,0.091052,24.257705,5.4834e-130,1.7474e-128,"""HGNC:37133"""
"""A1CF""","""ENSG00000148584""",null,"""UBERON_0002190""","""all_cohorts_Neuroblastoma_v_Adipose_-_Subcutaneous""","""Adipose - Subcutaneous""",663,0.35,0.01,4.084885,0.792896,0.201907,3.927042,0.000086,0.000167,"""HGNC:24086"""
"""A2M""","""ENSG00000175899""",null,"""UBERON_0002190""","""all_cohorts_Neuroblastoma_v_Adipose_-_Subcutaneous""","""Adipose - Subcutaneous""",663,116.32,715.05,58978.396,-0.713397,0.077601,-9.193122,3.8160e-20,1.6971e-19,"""HGNC:7"""
"""A2M-AS1""","""ENSG00000245105""",null,"""UBERON_0002190""","""all_cohorts_Neuroblastoma_v_Adipose_-_Subcutaneous""","""Adipose - Subcutaneous""",663,3.09,6.02,275.621445,0.153147,0.053443,2.865616,0.004162,0.00718,"""HGNC:27057"""


In [8]:
nbl_gc_all_merge = nbl_gc_all_merge.to_pandas()


In [ ]:
# do we want gene expression data for every tissue?